In [1]:
import numpy as np
from typing import NamedTuple

In [2]:
s='''##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^'''

def parse(s, part_two=False):
    m, d = s.split('\n\n')
    m = m.split('\n')
    if part_two:
        return DoubleMap(m), d.replace('\n', '')
    return Map(m), d.replace('\n', '')


In [21]:
class Point(NamedTuple):
    row: int
    col:int

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)

    
class Map:
    directions = {
        '>': Point(0, 1),
        '<': Point(0, -1),
        '^': Point(-1, 0),
        'v': Point(1, 0),
    }

    def __init__(self, s):
        self.w = len(s[0])
        self.h = len(s)

        self.points = {}

        for row, line in enumerate(s):
            for col, item in enumerate(line):
                if item == '.':
                    continue
                if item == '@':
                    self.robot_location = Point(row=row, col=col)
                self.points[Point(row=row, col=col)] = item

    def __repr__(self):
        grid = [['.'] * self.w for _ in range(self.h)]
        for point, item in self.points.items():
            grid[point.row][point.col] = item

        return '\n'.join(''.join(row) for row in grid)

    def move(self, point, direction):
        vector = Map.directions[direction]
        dest = point + vector
        
        item = self.points.get(point)
        
        if item == '#':
            return False
        elif item is None:
            return True

        if self.move(dest, direction):
            del self.points[point]
            self.points[dest] = item
            if item == '@':
                self.robot_location = dest
            return True

    def score(self):
        score = 0
        for p, item in self.points.items():
            if item == 'O':
                score += p.row * 100 + p.col
        return score

In [22]:
sample_map, sample_directions = parse(s)
sample_map

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

In [23]:
for d in sample_directions:
    sample_map.move(sample_map.robot_location, d)
sample_map

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########

In [24]:
sample_map.score()

10092

In [25]:
with open('input_files/15.txt') as f:
    raw = f.read()
big_map, directions = parse(raw)
  

In [26]:
big_map

##################################################
#..O.#.O.....O.#..O...O.O.O..O..O...O...O.....OO.#
#....OO........O........O..OO.O..O...##O.O#O..OOO#
#..O...O......O.O#...O.#......#O.#......O.....OO.#
#O.#O....OOOO.OO....O.......O.#.O..O##...O..#..#.#
#.O..O...OO......OOOO...........#OO.O..##.O.O.#..#
#.......O.O.OO...#.OOO..O.O..O.OO....O...O.....OO#
#O...O....O...OO#.....OO.O..O......#..#.O.OO.O...#
#.O.......O.#.#..#.O.......OO..O.....O#.OO.O...OO#
#OO.#..#..#...O.....O.O.............OO.OOO.O.....#
##O........O..........O.O#O..O.......O.O.OO.O....#
#OO...O.O...OO...OO......O...........#..O.#O.#O..#
#.O..#..#...O.....O.O.......OOO..O.OO.....O.O.O.O#
#.O#....O..O..O..#.......O.OO............O..O.#..#
##O..#...OO..O...O..#.....O....O..O.OO....##..O.O#
#.O.O...OOO...#OOO.O..O.O..O.....#..OOOO..O..O...#
##..##O..#.OO.#.....O#..O.O#O.O.O.O.....#..OO..#.#
#O...O.O.O##..O........O..O..........O...O.O.O.#.#
#.O..OO.O.#.#.#....OO.........O..O#.OO......#..OO#
##O..O.....O.OO..O.O.O#O....OOO

In [27]:
for d in directions:
    big_map.move(big_map.robot_location, d)
print("Score:", big_map.score())
big_map

Score: 1360570


##################################################
#.O..#OOOOO....#O..OOO.......OO...O.O..OOO.O.OOO.#
#.O..OOO....O....OO.OO.....O.....O...##..O#...OOO#
#O.........O.....#..O..#......#..#OO..........OOO#
#O.#.....O........OO...OOOO...#.OO..##O..OO.#..#O#
#........O.O......O...O..O.....O#O...O.##.....#OO#
#OO........O...OO#OOO.........OOO..........OOOOOO#
#OO...........OO#OOOO..........O...#..#O.......OO#
#OO.O..O....#.#..#OOO.................#O........O#
#OO.#..#..#...O...OO..........................O.O#
##O.........OOO....OO....#......O.O.O............#
#O...O..O..OOO...OO..........O.O..OOO#..O.#.O#O..#
#O...#O.#.O..O...OOO..........O..........O..OOOOO#
#.O#..@...O.OOOO.#OOO.O..OOOO......O.....O...O#..#
##O..#.....O.......O#......O.OO.OO.OO....O##.OOO.#
#OOO.....O....#......OO...O..OOOO#......O....OOOO#
##..##...#OOO.#..O...#..O..#.O.O.O.....O#....OO#.#
#O..O.O.OO##O....O..O...................OOOO...#.#
#O..O..O..#.#.#.....O.....OO...OO.#O.O......#...O#
##.....O........OOO.OO#O.OO...O

In [18]:
class DoubleMap(Map):
    def __init__(self, s):
        self.w = len(s[0]) * 2
        self.h = len(s)
    
        self.points = {}

        for row, line in enumerate(s):
            for col, item in enumerate(line):
                if item != '.':
                    if item == '@':
                        self.robot_location = Point(row=row, col=col*2)
                        self.points[Point(row=row, col=col*2)] = item
                    elif item=='O':
                        self.points[Point(row=row, col=col*2)] = '['
                        self.points[Point(row=row, col=col*2+1)] = ']'
                    else:
                        self.points[Point(row=row, col=col*2)] = item
                        self.points[Point(row=row, col=col*2+1)] = item

        
    def move(self, points, direction):
        vector = Map.directions[direction]
        points = set(points)

        if any(self.points.get(point) == '#' for point in points):
            return False
        if all(self.points.get(point) is None for point in points):
            return True

        if vector in (Point(-1, 0), Point(1, 0)):
            # ony do this for vertical movements horizontal movements 
            # take care of themselves and cause recursion errors here
            additional_points = set()
            for point in points:
                if self.points.get(point) == '[':
                    additional_points.add(point + Point(0, 1))
                if self.points.get(point) == ']':
                    additional_points.add(point + Point(0, -1))
            
            points.update(additional_points)
        
        # The None values were useful above to see if the space was free
        # But we don't want to propogate them
        points = [point for point in points if self.points.get(point) is not None]        
        dests = set([point + vector for point in points])
        
        if self.move(dests, direction):
            for point in points:
                item = self.points[point]
                del self.points[point]
                self.points[point + vector] = item
                if item == '@':
                    self.robot_location = point + vector
            return True
            
    def score(self):
        score = 0
        for p, item in self.points.items():
            if item == '[':
                score += p.row * 100 + p.col
        return score

In [19]:
sample_map, sample_directions = parse(s, part_two=True)

print(sample_map.robot_location)
sample_map

Point(row=4, col=8)


####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################

In [20]:
count = 0
for d in sample_directions:
    count += 1
    sample_map.move([sample_map.robot_location], d)
print("score:", sample_map.score())
sample_map

score: 9021


####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################

In [13]:
with open('input_files/15.txt') as f:
    raw = f.read()
big_map, directions = parse(raw, part_two=True)
big_map

####################################################################################################
##....[]..##..[]..........[]..##....[]......[]..[]..[]....[]....[]......[]......[]..........[][]..##
##........[][]................[]................[]....[][]..[]....[]......####[]..[]##[]....[][][]##
##....[]......[]............[]..[]##......[]..##............##[]..##............[]..........[][]..##
##[]..##[]........[][][][]..[][]........[]..............[]..##..[]....[]####......[]....##....##..##
##..[]....[]......[][]............[][][][]......................##[][]..[]....####..[]..[]..##....##
##..............[]..[]..[][]......##..[][][]....[]..[]....[]..[][]........[]......[]..........[][]##
##[]......[]........[]......[][]##..........[][]..[]....[]............##....##..[]..[][]..[]......##
##..[]..............[]..##..##....##..[]..............[][]....[]..........[]##..[][]..[]......[][]##
##[][]..##....##....##......[]..........[]..[]..........................[][]..[][][]..[]...

In [14]:
for d in directions:
    big_map.move([big_map.robot_location], d)

print("Score:", big_map.score())
big_map

Score: 1381446


####################################################################################################
##....[]..##..[]..........[]..##....[]....[][]..[]..[]...[][].[][]......[]......[][]........[][]..##
##........[][]................[]........[]......[]..[]........[]..[]......####[][][]##[]....[][][]##
##....[]......[]............[]..[]##....[]....##[].[].......##[]..##[][]....[].....[][][]....[][].##
##[]..##[]........[][][][]..[][]........[]..................##...[].....####........[][]##....##[]##
##..[]....[]......[][]..........[][][][]........................##[]......[]..####..[][][]..##..[]##
##..............[]..[]..[][]......##..[]........[]...............[][].....[]..[][]...[].[]..[][][]##
##[]......[]........[]......[][]##....[][]..[]..[]...............[]...##....##[][].......[].[][][]##
##..[]..............[]..##..##....##........[]..........................[][]##...........[]...[][]##
##[][]..##....##....##......[]....[].......[]......[]..[][]................[][]............